In [1]:
from bs4 import BeautifulSoup
import os
import pandas as pd
import numpy as np
import re
import random
from random import randint
import time #delay
from selenium import webdriver
from selenium.webdriver.common.by import By

### Scrape general reference info

In [ ]:

driver = webdriver.Edge() #Version 121.0.2277.98 (Official build) (arm64) , Edge (Windows)
driver.get("https://elelong.kehakiman.gov.my/BidderWeb/Home/Index")

#previous_page=driver.find_elements(By.CSS_SELECTOR, ".fa.fa-backward")
next_page=driver.find_elements(By.CSS_SELECTOR, ".fa.fa-forward")

df=pd.DataFrame()

counter=0

while counter < 100:
    
    soup=BeautifulSoup(driver.page_source)

    elements=soup.find_all('div',{'class':'auction-result-cell thumbnail'})

    for element in elements:
        auction_list=[]
        auction_list.append('https://elelong.kehakiman.gov.my'+element.find('a', {'class':'btn btn-primary aIamInterested'}).get('href'))
        for data in element.find('table', {'class':'table table-noline auction-result-info'}).find_all('tr'):
            auction_list.append(data.find('td', {'style' : 'font-weight: bold;'}).text)
        
        row=pd.DataFrame(columns=['link','Auction date/time','Land Used','State','District','Reserved Price (RM)','Tenure'],data=[auction_list])
        
        df=pd.concat([df, row])
        
    time.sleep(random.randint(3,5)) #random time sleep between 3-5 sec
    
    next_page[0].click()
    counter += 1

# df.to_csv('data-elelong-pt3.csv', index=False)
# df.to_csv('data-elelong-pt3.csv', index=False)
df.to_csv('data-elelong-pt3.csv', index=False)

### Create reference data

In [10]:
os.chdir('/Users/izard/Documents/Github/kehakiman-scraper/elelong')
df1=pd.read_csv('data-elelong-pt1.csv')
df2=pd.read_csv('data-elelong-pt2.csv')
df3=pd.read_csv('data-elelong-pt3.csv')
df=pd.concat([df1,df2,df3])
df

,link,Auction date/time,Land Used,State,District,Reserved Price (RM),Tenure
0,https://elelong.kehakiman.gov.my/BidderWeb/Hom...,"22/02/2024, 09:00:00 am",Building,Wilayah Persekutuan Kuala Lumpur,Daerah Kuala Lumpur,"291,600.00",Leasehold
1,https://elelong.kehakiman.gov.my/BidderWeb/Hom...,"22/02/2024, 09:00:00 am",Building,Wilayah Persekutuan Kuala Lumpur,Kuala Lumpur,"200,000.00",Leasehold
2,https://elelong.kehakiman.gov.my/BidderWeb/Hom...,"22/02/2024, 09:00:00 am",Building,Kedah,Kuala Muda,"135,000.00",Freehold
3,https://elelong.kehakiman.gov.my/BidderWeb/Hom...,"22/02/2024, 09:00:00 am",Building,Kedah,Kuala Muda,"340,000.00",Freehold
4,https://elelong.kehakiman.gov.my/BidderWeb/Hom...,"22/02/2024, 09:00:00 am",Building,Kedah,Kulim,"306,000.00",Freehold
...,...,...,...,...,...,...,...
1890,https://elelong.kehakiman.gov.my/BidderWeb/Hom...,"25/06/2024, 09:00:00 am",Building,Kedah,Kuala Muda,"150,000.00",Freehold
1891,https://elelong.kehakiman.gov.my/BidderWeb/Hom...,"17/07/2024, 11:00:00 am",Building,Johor,Johor Bahru,"610,000.00",Freehold
1892,https://elelong.kehakiman.gov.my/BidderWeb/Hom...,"17/07/2024, 11:00:00 am",Building,Johor,Batu Pahat,"1,000,000.00",Freehold
1893,https://elelong.kehakiman.gov.my/BidderWeb/Hom...,"15/08/2024, 09:00:00 am",Building,Selangor,Petaling,"630,000.00",Freehold


In [ ]:
df.reset_index(drop=True).drop_duplicates()

In [ ]:
import matplotlib


df.groupby('State').count()['link'].plot(kind='bar',figsize=(15,5))

### Scrape additional detail information

In [ ]:
from bs4 import BeautifulSoup
import os
import pandas as pd
import numpy as np
import re
import random
from random import randint
import time #delay
from selenium import webdriver
from selenium.webdriver.common.by import By

df_ref=pd.DataFrame()

df1=pd.read_csv('data-elelong-pt1.csv')
df2=pd.read_csv('data-elelong-pt2.csv')
df3=pd.read_csv('data-elelong-pt3.csv')
df=pd.concat([df1,df2,df3])

for link in df['link'].to_list():
    
    try:
        
        driver = webdriver.Edge()
        driver.get(link)
        soup=BeautifulSoup(driver.page_source)

        infos=[]
        columns=[]
        data_list=[]

        for info in soup.find('div',{'class':'tab-content clearfix'}).find_all('div',{'class':'col-xs-12 col-sm-12'})[0:7]:
            dets=info.text.strip().replace('\n', '').split(':')
            #print(dets)
            columns.append(dets[0])
            data_list.append(dets[1])
            
        for info in soup.find('strong',{'id':'prop-field-addr'}):  
            dets=['Address',info.text.strip()]
            #print(dets)
            columns.append(dets[0])
            data_list.append(dets[1])
            
        for i in list(range(28)):
            info=soup.find('div',{'class':'tab-content clearfix'}).find_all('div',{'class':'col-xs-12 col-sm-12'})[i].find_all('div',{'class':'col-xs-12 col-sm-12'})
            if len(info)==0:
                continue    
            else:
                infos.append(info)
                    
        infos=infos[-1]

        for detail in infos:
            dets=detail.text.strip().replace('\n', '').replace('Built Up (Sf)','Built Up (Sf):').split(':')
            columns.append(dets[0])
            data_list.append(dets[1])
            #print(dets)

        for script in soup.find_all('script'):
            results = re.search(r"https://efs.kehakiman.gov.my/EFSWeb/DocDownloader.aspx\?DocumentID.+true", script.text)
            if results:
                columns.append('link')
                data_list.append(link)
                columns.append('Document')
                data_list.append(results.group(0))
                
        row=pd.DataFrame(columns=columns,data=[data_list])
        df_ref=pd.concat([df_ref,row])
        
        time.sleep(randint(1,5))
        print(link,flush=True)
        driver.close()
        
    except:
        print('error',flush=True)
        #os.system('clear')
        pass
    
df_ref.to_csv('data-ref-elelong.csv',index=False)


### Download *.pdf based on elelong data ref

In [ ]:
# download pdf

os.chdir('/Users/izard/Documents/pdf/') # switch to folder where the pdf files to be kept

import requests


for link in pd.read_csv('/Users/izard/Documents/Github/kehakiman-scraper/elelong/data-ref-elelong.csv')['Document'].to_list():
    print(link)
    try:
        r=requests.get(link,allow_redirects=True)
        open(link.replace('https://efs.kehakiman.gov.my/EFSWeb/DocDownloader.aspx?DocumentID=','').replace('&Inline=true','')+'.pdf', 'wb').write(r.content)
    except:
        continue
    
# only 1,873 pdf available to download compared to 5,880 links , due to some links would be expired within few days after published , need to immedietly download the pdf once the detailed link obtained

### Convert pdf to text

In [ ]:
# start tika server , check at http://localhost:9998/

os.chdir('/Users/izard/Documents/Github/kehakiman-scraper/') # change directory to where tika-server-standard-2.6.0.jar is located

! java -jar tika-server-standard-2.6.0.jar

In [7]:
import os
import pandas as pd
import tika
from tika import parser

os.chdir('/Users/izard/Documents/pdf/')
downloaded_files=os.listdir() # get list of downloaded files

os.chdir('/Users/izard/Documents/Github/kehakiman-scraper/')

tika.initVM()
print('start..')

pdf_to_text=pd.DataFrame(columns=['Document'],data=downloaded_files)

try:
    os.chdir('/Users/izard/Documents/pdf/') #point where the pdf file located
    print('parsing...')
    pdf_to_text['Document_Text'] = pdf_to_text['Document'].apply(lambda x: parser.from_file(x)['content'])
    
except:
    print('unsuccess')
    pass


start..
parsing...


In [8]:
pdf_to_text

,Document,Document_Text
0,002573b5-1932-474a-8351-0cc3d4447ab6.pdf,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
1,002a1383-a392-4826-ace0-206d15e590e2.pdf,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
2,004a85b8-67de-46f2-b442-e308c480792f.pdf,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
3,004d7cc5-dd16-482d-9808-e1e9745ed447.pdf,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
4,006876ba-1f44-4862-942a-f200dca9b9b7.pdf,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
...,...,...
1868,ffd75cdd-f65e-4b01-8c44-a19af92cd764.pdf,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
1869,ffdb1a71-f400-45c6-bdd6-0b9485538d9e.pdf,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
1870,ffe600be-64b1-4ae0-87fb-118b9cf09f77.pdf,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
1871,fff56f27-aef5-4d99-863c-dd0334e04cbe.pdf,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...


In [13]:
pdf_to_text['Document']='https://efs.kehakiman.gov.my/EFSWeb/DocDownloader.aspx?DocumentID='+pdf_to_text['Document'].str.replace(r'\.pdf$','',regex=True)+'&Inline=true'
pdf_to_text.to_csv('/Users/izard/Documents/pdf_to_text_raw.csv', index=False)

In [12]:
df = pd.read_csv('data-ref-elelong.csv')
df

,Case Number,Proceeding ID,Reserved Price (RM),Number of Properties in Auction,Auction Date/Time,Number of Auction,Deposit (Refundable),Address,Lot Number,Parcel Number,...,Tenure,Built Up (Sf),Land Size (ft²),Land Size (m²),Reserved Area (Malay Reserve/Lot Bumi),Caveat,link,Document,Land Size (acre),Land Size (hectare)
0,WA-38-1688-09/2023,535ceac1-a75c-4c01-9a47-8859f5906c30,"RM 260,000.00",1,"05/03/2024, 09",1,"RM 26,000.00","Unit No. 26B-3, Faber Indah Apartment, Jalan D...",Lot 35287,119,...,Freehold,517,517.000000,48.000000,NaN,NaN,https://elelong.kehakiman.gov.my/BidderWeb/Hom...,https://efs.kehakiman.gov.my/EFSWeb/DocDownloa...,NaN,NaN
1,WA-38-1847-10/2023,0f8b763c-3a29-42e8-9b7c-22ed422d095b,"RM 1,800,000.00",1,"05/03/2024, 09",1,"RM 180,000.00","No. 32,, Jalan Daun Inai 17, Sunway SPK Damans...",PT 20133,NaN,...,Freehold,0,"2,239.000000",208.000000,NaN,NaN,https://elelong.kehakiman.gov.my/BidderWeb/Hom...,https://efs.kehakiman.gov.my/EFSWeb/DocDownloa...,NaN,NaN
2,WA-38-1869-10/2023,4258f509-b06f-4b0a-af3e-a5486c537eab,"RM 1,300,000.00",1,"05/03/2024, 09",1,"RM 130,000.00","Unit No. C-15-5, Kiara 9 Residency, No. 22, Ja...",Lot 68012,46,...,Freehold,0,"2,691.000000",250.000000,NaN,NaN,https://elelong.kehakiman.gov.my/BidderWeb/Hom...,https://efs.kehakiman.gov.my/EFSWeb/DocDownloa...,NaN,NaN
3,WA-38-2188-11/2023,6d521508-4cca-449c-8962-e83e56f2f7db,"RM 220,000.00",1,"05/03/2024, 09",1,"RM 22,000.00","UNIT NO C-12-20, APARTMENT SRI RAKYAT, JALAN 1...",LOT 101147,M3,...,Freehold,0,667.000000,62.000000,NaN,NaN,https://elelong.kehakiman.gov.my/BidderWeb/Hom...,https://efs.kehakiman.gov.my/EFSWeb/DocDownloa...,NaN,NaN
4,WA-38-2199-11/2023,0d9b1156-bea7-4b4a-a254-adf7a352c164,"RM 500,000.00",1,"05/03/2024, 09",1,"RM 50,000.00","Unit No. E1A-15-9, Block A, The Tamarind, Jala...",Lot 285 Seksyen 84,146,...,Freehold,0,"1,109.000000",103.000000,NaN,NaN,https://elelong.kehakiman.gov.my/BidderWeb/Hom...,https://efs.kehakiman.gov.my/EFSWeb/DocDownloa...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5427,KB-38-1200-12/2023,a66862b2-ff35-4583-bd94-ec7624dea162,"RM 150,000.00",1,"25/06/2024, 09",1,"RM 15,000.00","NO. 1374, LORONG KEMPAS 3/7C, TAMAN KEMPAS, 08...",LOT 30415 SEKSYEN 67,NaN,...,Freehold,0,"1,238.000000",115.000000,NaN,KAVEAT PERSENDIRIAN ATAS TANAH OLEH AMBANK ISL...,https://elelong.kehakiman.gov.my/BidderWeb/Hom...,https://efs.kehakiman.gov.my/EFSWeb/DocDownloa...,NaN,NaN
5428,JA-38-1011-06/2023,484ffd4e-d700-4ff1-8102-84102e8dab58,"RM 610,000.00",1,"17/07/2024, 11",1,"RM 61,000.00","NO. 49, JALAN BESTARI 11/4, TAMAN NUSA BESTARI...",LOT 121495,NaN,...,Freehold,0,"1,302.000000",121.000000,NaN,Kaveat Persendirian atas Tanah oleh SRI NANTHA...,https://elelong.kehakiman.gov.my/BidderWeb/Hom...,https://efs.kehakiman.gov.my/EFSWeb/DocDownloa...,NaN,NaN
5429,JA-38-652-04/2023,daeec663-2fdc-435a-990b-f46129662f0b,"RM 1,000,000.00",1,"17/07/2024, 11",1,"RM 100,000.00","No. 3, Jalan Limau Bali, Taman Pantai, 83000, ...",PTD 1609,NaN,...,Freehold,0,"5,169.990000",480.310000,NaN,1.\tKaveat Persendirian atas Tanah Perserahan No,https://elelong.kehakiman.gov.my/BidderWeb/Hom...,https://efs.kehakiman.gov.my/EFSWeb/DocDownloa...,NaN,NaN
5430,BA-38-1289-07/2023,656e2f07-6602-4c4b-b24b-8defe42c3939,"RM 630,000.00",1,"15/08/2024, 09",1,"RM 63,000.00","No 42, Jalan USJ 12/2, USJ 12, 47630, UEP Suba...",Lot 33108,NaN,...,Freehold,0,"1,195.000000",111.000000,NaN,NaN,https://elelong.kehakiman.gov.my/BidderWeb/Hom...,https://efs.kehakiman.gov.my/EFSWeb/DocDownloa...,NaN,NaN


In [15]:
train=pd.merge(df, pdf_to_text, how='left',left_on='Document', right_on='Document')
train['Text_Len']=train['Document_Text'].str.len()
train['Text_Ext_Method']='Tika 2.6.0'
train.to_csv('train.csv', index=False)
train

,Case Number,Proceeding ID,Reserved Price (RM),Number of Properties in Auction,Auction Date/Time,Number of Auction,Deposit (Refundable),Address,Lot Number,Parcel Number,...,Land Size (m²),Reserved Area (Malay Reserve/Lot Bumi),Caveat,link,Document,Land Size (acre),Land Size (hectare),Document_Text,Text_Len,Text_Ext_Method
0,WA-38-1688-09/2023,535ceac1-a75c-4c01-9a47-8859f5906c30,"RM 260,000.00",1,"05/03/2024, 09",1,"RM 26,000.00","Unit No. 26B-3, Faber Indah Apartment, Jalan D...",Lot 35287,119,...,48.000000,NaN,NaN,https://elelong.kehakiman.gov.my/BidderWeb/Hom...,https://efs.kehakiman.gov.my/EFSWeb/DocDownloa...,NaN,NaN,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,20118.0,Tika 2.6.0
1,WA-38-1847-10/2023,0f8b763c-3a29-42e8-9b7c-22ed422d095b,"RM 1,800,000.00",1,"05/03/2024, 09",1,"RM 180,000.00","No. 32,, Jalan Daun Inai 17, Sunway SPK Damans...",PT 20133,NaN,...,208.000000,NaN,NaN,https://elelong.kehakiman.gov.my/BidderWeb/Hom...,https://efs.kehakiman.gov.my/EFSWeb/DocDownloa...,NaN,NaN,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,20423.0,Tika 2.6.0
2,WA-38-1869-10/2023,4258f509-b06f-4b0a-af3e-a5486c537eab,"RM 1,300,000.00",1,"05/03/2024, 09",1,"RM 130,000.00","Unit No. C-15-5, Kiara 9 Residency, No. 22, Ja...",Lot 68012,46,...,250.000000,NaN,NaN,https://elelong.kehakiman.gov.my/BidderWeb/Hom...,https://efs.kehakiman.gov.my/EFSWeb/DocDownloa...,NaN,NaN,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,20627.0,Tika 2.6.0
3,WA-38-2188-11/2023,6d521508-4cca-449c-8962-e83e56f2f7db,"RM 220,000.00",1,"05/03/2024, 09",1,"RM 22,000.00","UNIT NO C-12-20, APARTMENT SRI RAKYAT, JALAN 1...",LOT 101147,M3,...,62.000000,NaN,NaN,https://elelong.kehakiman.gov.my/BidderWeb/Hom...,https://efs.kehakiman.gov.my/EFSWeb/DocDownloa...,NaN,NaN,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,20174.0,Tika 2.6.0
4,WA-38-2199-11/2023,0d9b1156-bea7-4b4a-a254-adf7a352c164,"RM 500,000.00",1,"05/03/2024, 09",1,"RM 50,000.00","Unit No. E1A-15-9, Block A, The Tamarind, Jala...",Lot 285 Seksyen 84,146,...,103.000000,NaN,NaN,https://elelong.kehakiman.gov.my/BidderWeb/Hom...,https://efs.kehakiman.gov.my/EFSWeb/DocDownloa...,NaN,NaN,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,20064.0,Tika 2.6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5427,KB-38-1200-12/2023,a66862b2-ff35-4583-bd94-ec7624dea162,"RM 150,000.00",1,"25/06/2024, 09",1,"RM 15,000.00","NO. 1374, LORONG KEMPAS 3/7C, TAMAN KEMPAS, 08...",LOT 30415 SEKSYEN 67,NaN,...,115.000000,NaN,KAVEAT PERSENDIRIAN ATAS TANAH OLEH AMBANK ISL...,https://elelong.kehakiman.gov.my/BidderWeb/Hom...,https://efs.kehakiman.gov.my/EFSWeb/DocDownloa...,NaN,NaN,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,20484.0,Tika 2.6.0
5428,JA-38-1011-06/2023,484ffd4e-d700-4ff1-8102-84102e8dab58,"RM 610,000.00",1,"17/07/2024, 11",1,"RM 61,000.00","NO. 49, JALAN BESTARI 11/4, TAMAN NUSA BESTARI...",LOT 121495,NaN,...,121.000000,NaN,Kaveat Persendirian atas Tanah oleh SRI NANTHA...,https://elelong.kehakiman.gov.my/BidderWeb/Hom...,https://efs.kehakiman.gov.my/EFSWeb/DocDownloa...,NaN,NaN,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,20604.0,Tika 2.6.0
5429,JA-38-652-04/2023,daeec663-2fdc-435a-990b-f46129662f0b,"RM 1,000,000.00",1,"17/07/2024, 11",1,"RM 100,000.00","No. 3, Jalan Limau Bali, Taman Pantai, 83000, ...",PTD 1609,NaN,...,480.310000,NaN,1.\tKaveat Persendirian atas Tanah Perserahan No,https://elelong.kehakiman.gov.my/BidderWeb/Hom...,https://efs.kehakiman.gov.my/EFSWeb/DocDownloa...,NaN,NaN,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,20339.0,Tika 2.6.0
5430,BA-38-1289-07/2023,656e2f07-6602-4c4b-b24b-8defe42c3939,"RM 630,000.00",1,"15/08/2024, 09",1,"RM 63,000.00","No 42, Jalan USJ 12/2, USJ 12, 47630, UEP Suba...",Lot 33108,NaN,...,111.000000,NaN,NaN,https://elelong.kehakiman.gov.my/BidderWeb/Hom...,https://efs.kehakiman.gov.my/EFSWeb/DocDownloa...,NaN,NaN,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,19985.0,Tika 2.6.0
